SO 
Here is how I'm gonna do this
as a first try, without any preprocessing, simply combine all the data into a single
dataframe, regardless of threshold, just to nail down how the data needs to be structured

one by one, read in the csv files, creating one line in the data frame for every 30 frames
then write the identity in terms of video, bramble-or-rose, startframe, id, extracted body-length, as the 
first 5 elements in the row. The 6th entry is then the size class, so we can use that as a
label. without any smoothing, angles, or speeds, the shape of the data-frame will then be

(n, 30 * num_key_points + 6)

to run dimensionality reduction on that data, simply repeat the setup above:

In [ ]:
import pickle
import os
import pandas as pd
from umap import UMAP
from sklearn.manifold import TSNE
import plotly.express as px
from os import listdir
from os.path import isfile, join

In [ ]:
input_folder = "I:/WOLO/WOLO/EAEAAO/EXTRACTED_POSE_FEATURE_VECTORS"
df_ant_poses_list =  [os.path.join(input_folder,f) for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

df_ant_poses_dfs = [pd.read_pickle(filename) for filename in df_ant_poses_list]

In [ ]:
# stack the DataFrames
df_ant_poses = pd.concat(df_ant_poses_dfs, ignore_index=True, axis=0)

features = df_ant_poses.loc[:, 'raw_pose_0':] # use all entries and exclude labels
labels =  df_ant_poses.loc[:, :"size_class"]

norm_features = (features-features.min())/(features.max()-features.min())

In [ ]:
print("extracted feature vector contains",norm_features.shape[0],"instances and", norm_features.shape[1], "features")
labels

In [ ]:
# as a test, run a 2D tSNE
tsne = TSNE(n_components=2, random_state=0, init="pca", learning_rate="auto", perplexity=50)
projections = tsne.fit_transform(norm_features)

In [ ]:
projections_and_labels = labels.copy()
projections_and_labels["x"] = projections[:,0]
projections_and_labels["y"] = projections[:,1]

projections_and_labels.to_pickle("tSNE_perplexity_50")

# now take the produced embedding and plot with different colour overlays to group by class, speed, or material
fig_size = px.scatter(
    projections_and_labels, x="x", y="y",
    color="size_class", labels={'color': 'size_class'},
    opacity=0.1,
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    width=600, height=500) 

fig_size.write_image("tSNE_fig_size.svg")
fig_size.write_html("tSNE_fig_size.html")

fig_speed = px.scatter(
    projections_and_labels, x="x", y="y",
    color="speed", labels={'color': 'speed'},
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    range_color=(0,50),
    opacity=0.1,
    width=600, height=500) 

fig_speed.write_image("tSNE_fig_speed.svg")
fig_speed.write_html("tSNE_fig_speed.html")

fig_material = px.scatter(
    projections_and_labels, x="x", y="y",
    color="material", labels={'color': 'material'},
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    opacity=0.1,
    width=600, height=500) 

fig_material.write_image("tSNE_fig_material.svg")
fig_material.write_html("tSNE_fig_material.html")

fig_size.show()
fig_speed.show()
fig_material.show()

In [ ]:
umap_2d = UMAP(n_components=2, init='pca')#, random_state=0)
proj_2d = umap_2d.fit_transform(norm_features)

projections_and_labels_UMAP = labels.loc[200000:300000,:].copy()
projections_and_labels_UMAP["x"] = proj_2d[:,0]
projections_and_labels_UMAP["y"] = proj_2d[:,1]

projections_and_labels_UMAP.to_pickle("UMAP_df")

fig_size_2d = px.scatter(
    projections_and_labels_UMAP, x="x", y="y",
    color="size_class", labels={'color': 'size_class'},
    opacity=0.1,
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    width=600, height=500) 

fig_speed_2d = px.scatter(
    projections_and_labels_UMAP, x="x", y="y",
    color="speed", labels={'color': 'speed'},
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    opacity=0.1,
    range_color=(0,50),
    width=600, height=500) 

fig_material_2d = px.scatter(
    projections_and_labels_UMAP, x="x", y="y",
    color="material", labels={'color': 'material'},
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    opacity=0.1,
    width=600, height=500) 

fig_size_2d.show()
fig_speed_2d.show()
fig_material_2d.show()

We can also load produced embeddings for further analysis (such as hdbscan clustering) and re-plotting

In [ ]:
embedding_loc = "Z:/home/EAEAAO/UMAP_2D_mean-normalisation/UMAP_df"
projections_and_labels_UMAP = pd.read_pickle(embedding_loc)

fig_size_2d = px.scatter(
    projections_and_labels_UMAP, x="x", y="y",
    color="size_class", labels={'color': 'size_class'},
    opacity=0.01,
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    width=1000, height=800) 

fig_size_2d.write_image("UMAP_fig_size.svg")
fig_size_2d.write_html("UMAP_fig_size.html")

fig_speed_2d = px.scatter(
    projections_and_labels_UMAP, x="x", y="y",
    color="speed", labels={'color': 'speed'},
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    opacity=0.01,
    range_color=(0,50),
    width=1000, height=800) 

fig_speed_2d.write_image("UMAP_fig_speed.svg")
fig_speed_2d.write_html("UMAP_fig_speed.html")

fig_material_2d = px.scatter(
    projections_and_labels_UMAP, x="x", y="y",
    color="material", labels={'color': 'material'},
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    opacity=0.01,
    width=1000, height=800) 

fig_material_2d.write_image("UMAP_fig_material.svg")
fig_material_2d.write_html("UMAP_fig_material.html")

#fig_size_2d.show()
#fig_speed_2d.show()
#fig_material_2d.show()

In [ ]:
import hdbscan
import numpy as np
clusterable_embedding = projections_and_labels_UMAP.loc[:,"x":"y"]

labels = hdbscan.HDBSCAN(
    min_samples=100,
    min_cluster_size=1000,
    cluster_selection_epsilon=0.03,
    alpha = 0.5,
    cluster_selection_method="leaf"
).fit_predict(clusterable_embedding)

print("Datapoints assigned to clusters:",len(labels[clustered]))
print("Datapoints unassigned:",len(labels[~clustered]))

In [ ]:
standard_embedding = projections_and_labels_UMAP.loc[:,"x":"y"].to_numpy()

clustered = (labels >= 0)
plt.scatter(standard_embedding[~clustered, 0],
            standard_embedding[~clustered, 1],
            color=(0.5, 0.5, 0.5),
            s=0.1,
            alpha=0.5)
plt.scatter(standard_embedding[clustered, 0],
            standard_embedding[clustered, 1],
            c=labels[clustered],
            s=0.1,
            cmap='Spectral');
plt.show()

In [ ]:
fig_class_2d = px.scatter(
    projections_and_labels_UMAP.loc[clustered,:], x="x", y="y",
    color=labels[clustered], labels={'color': "cluster"},
    opacity=0.1,
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    width=1000, height=800) 

fig_class_2d.write_image("UMAP_fig_class.svg")
fig_class_2d.write_html("UMAP_fig_class.html")

#fig_class_2d.show()

In [ ]:
fig_non_class_2d = px.scatter(
    projections_and_labels_UMAP.loc[~clustered,:], x="x", y="y",
    opacity=0.01, 
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    width=1000, height=800) 

fig_non_class_2d.write_image("UMAP_fig_unassigned.svg")
fig_non_class_2d.write_html("UMAP_fig_unassigned.html")

#fig_non_class_2d.show()

In [ ]:
2176
2904
274

at_frame = 156978

In [ ]:
projections_and_labels_UMAP[projections_and_labels_UMAP.id == 2176][projections_and_labels_UMAP.startframe > at_frame]

In [ ]:
projections_and_labels_UMAP[projections_and_labels_UMAP.id == 2904][projections_and_labels_UMAP.startframe > at_frame]

In [ ]:
projections_and_labels_UMAP[projections_and_labels_UMAP.id == 274][projections_and_labels_UMAP.startframe > at_frame]

In [ ]:
# plot clusters for specific materials

fig_bramble_2d = px.scatter(
    projections_and_labels_UMAP[projections_and_labels_UMAP.material=="bramble"], x="x", y="y",
    opacity=0.1,
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    width=1000, height=800) 

fig_bramble_2d.write_image("UMAP_fig_bramble.svg")
fig_bramble_2d.write_html("UMAP_fig_bramble.html")

fig_rose_2d = px.scatter(
    projections_and_labels_UMAP[projections_and_labels_UMAP.material=="rose"], x="x", y="y",
    hover_name="video", hover_data=["material","startframe","id","speed","size_class"],
    opacity=0.1,
    range_color=(0,50),
    width=1000, height=800) 

fig_rose_2d.write_image("UMAP_fig_rose.svg")
fig_rose_2d.write_html("UMAP_fig_rose.html")

#fig_rose_2d.show()
#fig_bramble_2d.show()

In [ ]:
fig_size_2d.update_xaxes(range=[5, 10])
fig_size_2d.update_yaxes(range=[-2, 4])

fig_size_2d.write_image("UMAP_fig_size_cropped.svg")

fig_speed_2d.update_xaxes(range=[5, 10])
fig_speed_2d.update_yaxes(range=[-2, 4])

fig_speed_2d.write_image("UMAP_fig_speed_cropped.svg")

fig_material_2d.update_xaxes(range=[5, 10])
fig_material_2d.update_yaxes(range=[-2, 4])

fig_material_2d.write_image("UMAP_fig_material_cropped.svg")